<a href="https://colab.research.google.com/github/pedrohortencio/Machine-Learning-Algorithms-in-the-Real-World/blob/main/Supervised%20Learning%20Tip%20to%20Tail/knn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<font color=red>This is a draft version and the notebook is incomplete and the statements do not match the data at this point. It will be fixed soon.</font>

# $k$-Nearest Neighbours ($k$-NN)

Welcome! In this Jupyter notebook, you will use scikit-learn to build and predict using the $k$-Nearest Neighbour (or $k$-NN) algorithm. We will also talk distance measures, a bit. But that's enough, let's jump into it.

## Before you start

- In order for the notebooks to function as intended, modify only between lines marked "### begin your code here (__ lines)." and "### end your code here.". 

- The line count is a suggestion of how many lines of code you need to accomplish what is asked.

- You should execute the cells (the boxes that a notebook is composed of) in order.

- You can execute a cell by pressing Shift and Enter (or Return) simultaneously.

- You should have completed the _Decision Trees_ Jupyter notebook before attempting this one as the concepts covered there are not repeated, for the sake of brevity.

## Loading the appropriate packages

Again, we are loading required packages. From scikit-learn, we will load the packages appropriate for k-NNs this time. 

In [ ]:
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

Let's turn off the scientific notation for floating point numbers.

In [ ]:
np.set_printoptions(suppress=True)

## Loading and examining the data

We will load our data from a CSV file and put it in a pandas an object of the `DataFrame` class.

This is the Iris flower dataset, a very famous dataset which was published in 1936 by studying three different species of the flower Iris: _Iris setosa_, _Iris versicolor_ and _Iris virginica_. Originally, the dataset has 150 examples which corresponds to 150 different Iris flowers measured (50 flowers from each species). The dataset has 4 features, the sepal length, sepal width, petal length and petal width of each flower in centimeters. However, for the purpose of ease of illustration, we have chosen only two of the features: the sepal length and sepal width.

* This was taken and modified from the Machine Learning dataset repository of School of Information and Computer Science of University of California Irvine (UCI):
 
> _Dua, D. and Graff, C. (2019). UCI Machine Learning Repository [http://archive.ics.uci.edu/ml]. Irvine, CA: University of California, School of Information and Computer Science._

In [ ]:
df = pd.read_csv('/content/data_knn.csv')

Let's take a look at our data as a table:

In [ ]:
df

,Sepal Length,Sepal Width,Species
0,5.1,3.5,Iris setosa
1,4.9,3.0,Iris setosa
2,4.7,3.2,Iris setosa
3,4.6,3.1,Iris setosa
4,5.0,3.6,Iris setosa
...,...,...,...
145,6.7,3.0,Iris virginica
146,6.3,2.5,Iris virginica
147,6.5,3.0,Iris virginica
148,6.2,3.4,Iris virginica


I'll do some verifications and changes in the dataset, to see if it makes a difference in the predictions.

In [ ]:
df2 = df.copy()

max_SL = df2['Sepal Length'].max()
#print(df2[df2['Sepal Length']==df['Sepal Length'].max()])
min_SL = df2['Sepal Length'].min()

max_SW = df2['Sepal Width'].max()
min_SW = df2['Sepal Width'].min()

print('Max and min sepal length: ', max_SL, min_SL)
print('Max and min sepal width: ', max_SW, min_SW)

Max and min sepal length:  7.9 4.3
Max and min sepal width:  4.4 2.0


In [ ]:
df2['Sepal Length'] = df2['Sepal Length'].map(lambda x: x/max_SL)
df2['Sepal Width'] = df2['Sepal Width'].map(lambda x: x/max_SW)
df2.head(10)

,Sepal Length,Sepal Width,Species
0,0.645570,0.795455,Iris setosa
1,0.620253,0.681818,Iris setosa
2,0.594937,0.727273,Iris setosa
3,0.582278,0.704545,Iris setosa
4,0.632911,0.818182,Iris setosa
5,0.683544,0.886364,Iris setosa
6,0.582278,0.772727,Iris setosa
7,0.632911,0.772727,Iris setosa
8,0.556962,0.659091,Iris setosa
9,0.620253,0.704545,Iris setosa


Because now our data has 2 features, we can use a single scatter plot to take a look at our data:

In [ ]:
fig = px.scatter(df, x="Sepal Length", y="Sepal Width", color="Species")
fig.show()

In [ ]:
#with my df2
fig = px.scatter(df2, x="Sepal Length", y="Sepal Width", color="Species")
fig.show()

Let's extract our data and targets as NumPy arrays `X` and `y`, from pandas data frame `df`. This time, for visualization purposes, we need our targets `y` to be integers instead of text, so we have to extract text labels as `y_text` and then transform them into integer labels `y`. Fortunately, we can do that with the `LabelEncoder` class that comes in scikit-learn:

In [ ]:
X = df.drop('Species', axis=1).to_numpy()
y_text = df['Species'].to_numpy()
y = LabelEncoder().fit_transform(y_text)

In [ ]:
#with df2
X2 = df2.drop('Species', axis=1).to_numpy()
y_text2 = df2['Species'].to_numpy()
y2 = LabelEncoder().fit_transform(y_text2)

Let's see our data `X`:

In [ ]:
X

array([[5.1, 3.5],
       [4.9, 3. ],
       [4.7, 3.2],
       [4.6, 3.1],
       [5. , 3.6],
       [5.4, 3.9],
       [4.6, 3.4],
       [5. , 3.4],
       [4.4, 2.9],
       [4.9, 3.1],
       [5.4, 3.7],
       [4.8, 3.4],
       [4.8, 3. ],
       [4.3, 3. ],
       [5.8, 4. ],
       [5.7, 4.4],
       [5.4, 3.9],
       [5.1, 3.5],
       [5.7, 3.8],
       [5.1, 3.8],
       [5.4, 3.4],
       [5.1, 3.7],
       [4.6, 3.6],
       [5.1, 3.3],
       [4.8, 3.4],
       [5. , 3. ],
       [5. , 3.4],
       [5.2, 3.5],
       [5.2, 3.4],
       [4.7, 3.2],
       [4.8, 3.1],
       [5.4, 3.4],
       [5.2, 4.1],
       [5.5, 4.2],
       [4.9, 3.1],
       [5. , 3.2],
       [5.5, 3.5],
       [4.9, 3.6],
       [4.4, 3. ],
       [5.1, 3.4],
       [5. , 3.5],
       [4.5, 2.3],
       [4.4, 3.2],
       [5. , 3.5],
       [5.1, 3.8],
       [4.8, 3. ],
       [5.1, 3.8],
       [4.6, 3.2],
       [5.3, 3.7],
       [5. , 3.3],
       [7. , 3.2],
       [6.4, 3.2],
       [6.9,

In [ ]:
X2

array([[0.64556962, 0.79545455],
       [0.62025316, 0.68181818],
       [0.59493671, 0.72727273],
       [0.58227848, 0.70454545],
       [0.63291139, 0.81818182],
       [0.6835443 , 0.88636364],
       [0.58227848, 0.77272727],
       [0.63291139, 0.77272727],
       [0.55696203, 0.65909091],
       [0.62025316, 0.70454545],
       [0.6835443 , 0.84090909],
       [0.60759494, 0.77272727],
       [0.60759494, 0.68181818],
       [0.5443038 , 0.68181818],
       [0.73417722, 0.90909091],
       [0.72151899, 1.        ],
       [0.6835443 , 0.88636364],
       [0.64556962, 0.79545455],
       [0.72151899, 0.86363636],
       [0.64556962, 0.86363636],
       [0.6835443 , 0.77272727],
       [0.64556962, 0.84090909],
       [0.58227848, 0.81818182],
       [0.64556962, 0.75      ],
       [0.60759494, 0.77272727],
       [0.63291139, 0.68181818],
       [0.63291139, 0.77272727],
       [0.65822785, 0.79545455],
       [0.65822785, 0.77272727],
       [0.59493671, 0.72727273],
       [0.

Let's check the shape of `X`as well:

In [ ]:
X.shape

(150, 2)

Let's do the same checks with the targets vector `y`:

In [ ]:
y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [ ]:
y2

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

...and the shape of `y`:

In [ ]:
y.shape

(150,)

Everything looks good.

## Splitting data

Again, let's split our data into training, validation and test sets. Let's use 60% (90 examples) for training, 20% for validation (30 examples) and the remaining 20% (30 examples) as test data.

In [ ]:
(X_train, X_vt, y_train, y_vt) = train_test_split(X, y, test_size=0.4, random_state=0)
(X_validation, X_test, y_validation, y_test) = train_test_split(X_vt, y_vt, test_size=0.5, random_state=0)

In [ ]:
(X_train2, X_vt2, y_train2, y_vt2) = train_test_split(X2, y2, test_size=0.4, random_state=0)
(X_validation2, X_test2, y_validation2, y_test2) = train_test_split(X_vt2, y_vt2, test_size=0.5, random_state=0)

## Building and visualizing a $k$-NN model

Now, let's build our $k$-NN model! We will create an object of the class `KNeighborsClassifier` and assign the name `knn` for the resulting object. Remember that we have to specify a number for $k$ which is called `n_neighbors` in scikit-learn implementation.

However, before doing that let's have a quick discussion about distance measures which greatly affect how our $k$-NN classifier performs: Remember that you have to use your knowledge of the data to come up with a distance measure that makes sense for the data you have. You also may have to scale different features by different weights to get a good diatnce measurement out of their combination, before sttempting to train and use a model. For example, if we have integer data, it may make sense to use a Manhattan distance measure. Or use Hamming distance for bit data.

In our data here, all our measure ments are of length and are in centimeters, so no adjustment is needed and a Euclidian distance measure actually makes a lot of sense, since it is measuring some kind of length and the unweighted combination is also sound. For `KNeighborsClassifier`, the default distance measure or `metric` (as the argument is called in scikit-learn) is the $L_p$ measure or the Minkowski distance (`metric=minkowski` in scikit-learn call) with $p=2$ (`p=2` in `KNeighborsClassifier` arguments), which is nothing other than the $L_2$ distance measure or the Eucliadian distance.

So, other than setting `n_neighbors` to a suitable value, we don't have to specify anything else. Let's start with a value of 1 for `n_neighbors`. 

You can see the documentation for `KNeighborsClassifier` here:

https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html

Go ahead and implement that now:

In [ ]:
### begin your code here (1 line).
knn = KNeighborsClassifier(n_neighbors=1)
### end your code here.

In [ ]:
### begin your code here (1 line).
knn_2 = KNeighborsClassifier(n_neighbors=1)
### end your code here.

Next, let's fit our `knn` to our `X_train` and `y_train`. This does nothing but store the training example as $k$-NN is "lazy": It does all calculations as prediction time and by measuring the distance from the operational datapoints provided (whose labels have to be predicted) to each of the training datapoints, finding the closest training datapoints to the operational points, looking at the labels for those closest training datapoints, and finding the majority class among them.

In [ ]:
### begin your code here (1 line).
knn.fit(X_train, y_train)
### end your code here.

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=1, p=2,
                     weights='uniform')

In [ ]:
### begin your code here (1 line).
knn_2.fit(X_train2, y_train2)
### end your code here.

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=1, p=2,
                     weights='uniform')

Again, you will get a summary for the model:

> KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
>            metric_params=None, n_jobs=None, n_neighbors=1, p=2,
>            weights='uniform')

Now, let's visualize our $k$-NN model. We are using plotly heatmaps (which are a bit more tedious to produce at this moment in time as they have not been ported to plotly express yet) to show regions where points will be predicted in different classes (this will take some time) and we will overlay a scatter plot of training points on top of that:

In [ ]:
detail_steps = 500

(x_vis_0_min, x_vis_0_max) = (X[:, 0].min() - 0.5, X[:, 0].max() + 0.5)
(x_vis_1_min, x_vis_1_max) = (X[:, 1].min() - 0.5, X[:, 1].max() + 0.5)

x_vis_0_range = np.linspace(x_vis_0_min, x_vis_0_max, detail_steps)
x_vis_1_range = np.linspace(x_vis_1_min, x_vis_1_max, detail_steps)

(XX_vis_0, XX_vis_1) = np.meshgrid(x_vis_0_range, x_vis_1_range)
X_vis = np.c_[XX_vis_0.reshape(-1), XX_vis_1.reshape(-1)]

yhat_vis = knn.predict(X_vis)
YYhat_vis = yhat_vis.reshape(XX_vis_0.shape)

region_colorscale = [
                     [0.0, 'rgb(199, 204, 249)'],
                     [0.5, 'rgb(235, 185, 177)'],
                     [1.0, 'rgb(159, 204, 186)']
                    ]
points_colorscale = [
                     [0.0, 'rgb(99, 110, 250)'],
                     [0.5, 'rgb(239, 85, 59)'],
                     [1.0, 'rgb(66, 204, 150)']
                    ]
fig2 = go.Figure(
                data=[
                      go.Heatmap(x=x_vis_0_range,
                                 y=x_vis_1_range,
                                 z=YYhat_vis,
                                 colorscale=region_colorscale,
                                 showscale=False),
                      go.Scatter(x=df['Sepal Length'], 
                                 y=df['Sepal Width'],
                                 mode='markers',
                                 text=df['Species'],
                                 name='',
#                                 showscale=False,
                                 marker=dict(
                                             color=y,
                                             colorscale=points_colorscale
                                            )
                                )
                     ],
                     layout=go.Layout(
                                      xaxis=dict(title='Sepal Length'),
                                      yaxis=dict(title='Sepal Width')
                                     )
               )
fig2.show()

## Model selection and asessment

Evaluation time! Let's first see how does or $k$-NN does on training data. We expect to get an accuracy result of near 100% or 1.0. Not exactly 100% because the algorihthm in scikit-learn does not count the same point as a neighbour. Otherwise, we would have had a perfect 100% since the closest single point to each training point in the training points is that point itself and the classes predicted will match that of the training targets. So go ahead and calculate `yhat_train` using `knn` and `X_train`:

In [ ]:
### begin your code here (1 line).
yhat_train = knn.predict(X_train)
### end your code here.

In [ ]:
### begin your code here (1 line).
yhat_train_2 = knn_2.predict(X_train2)
### end your code here.

Now, to measure the accuracy from `yhat_train` and `y_train`:

In [ ]:
accuracy_score(yhat_train, y_train)

0.9444444444444444

In [ ]:
accuracy_score(yhat_train_2, y_train2)

0.9444444444444444

It is ??.??%. It is as we expected. And we also expect the accuracy on validation points not be that high, since we are probably overfitting by using $k=1$. Put in the line that generates `yhat_validation`, so we can measure the accuracy:

In [ ]:
### begin your code here (1 line).
yhat_validation = knn.predict(X_validation)
### end your code here.
accuracy_score(yhat_validation, y_validation)

0.7

In [ ]:
### begin your code here (1 line).
yhat_validation_2 = knn_2.predict(X_validation2)
### end your code here.
accuracy_score(yhat_validation_2, y_validation2)

0.6666666666666666

Okay, we got ??.??% which has a big gap with the accuracy on training data.

Let's use a higher value of $k$ or `n_neighbors`. Let's use 3 and see what happens. Go ahead a make a new model `knn3` and provide `X_train` and `y_train` to it afterwards using the methd `fit`:

In [ ]:
### begin your code here (2 lines).
knn3 = KNeighborsClassifier(n_neighbors = 3)
knn3.fit(X_train, y_train)
### end your code here.

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=3, p=2,
                     weights='uniform')

In [ ]:
### begin your code here (2 lines).
knn3_2 = KNeighborsClassifier(n_neighbors = 3)
knn3_2.fit(X_train2, y_train2)
### end your code here.

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=3, p=2,
                     weights='uniform')

Our model summary should indicate `n_neighbors=3` now:

> KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
>            metric_params=None, n_jobs=None, **n_neighbors=3**, p=2,
>            weights='uniform')

We can visualize the model with $k=3$:

In [ ]:
yhat3_vis = knn3.predict(X_vis)
YYhat3_vis = yhat3_vis.reshape(XX_vis_0.shape)

fig3 = fig2
fig3['data'][0]['z'] = YYhat3_vis

fig3.show()

You can see the regions are smoother and less "patchy".

Now, predict `yhat_train3` with this new `knn3`, so we can the accuracy on training data:

In [ ]:
### begin your code here (1 line).
yhat_train3 = knn3.predict(X_train)
### end your code here.
accuracy_score(yhat_train3, y_train)

0.8888888888888888

In [ ]:
### begin your code here (1 line).
yhat_train3_2 = knn3_2.predict(X_train2)
### end your code here.
accuracy_score(yhat_train3_2, y_train2)

0.8777777777777778

We got ??.??% This time we did not predict to near 100% accuracy since the training points surrounding a training point may have different labels. On to validation accuracy then. Let's predict `yhat_validation3` and we can the accuracy on validation data:

In [ ]:
### begin your code here (1 line).
yhat_validation3 = knn3.predict(X_validation)
### end your code here.
accuracy_score(yhat_validation3, y_validation)

0.6333333333333333

In [ ]:
### begin your code here (1 line).
yhat_validation3_2 = knn3_2.predict(X_validation2)
### end your code here.
accuracy_score(yhat_validation3_2, y_validation2)

0.6666666666666666

We get a better accuracy on validation points (??.??%), again as expected, since we are probably overfitting less. Let's try a $k=5$ as well. Make a new model `knn5` with `n_neighbors=5` and provide the data to it via `fit`:

In [ ]:
### begin your code here (2 lines).
knn5 = KNeighborsClassifier(n_neighbors = 5)
knn5.fit(X_train, y_train)
### end your code here.

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [ ]:
### begin your code here (2 lines).
knn5_2 = KNeighborsClassifier(n_neighbors = 5)
knn5_2.fit(X_train2, y_train2)
### end your code here.

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

You should get:

> KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
>            metric_params=None, n_jobs=None, **n_neighbors=5**, p=2,
>            weights='uniform')
           
We can visualize the model with $k=5$ as well:

In [ ]:
yhat5_vis = knn5.predict(X_vis)
YYhat5_vis = yhat3_vis.reshape(XX_vis_0.shape)

fig4 = fig2
fig4['data'][0]['z'] = YYhat5_vis

fig4.show()

You can see the regions are even more smooth.

Next, let's evaluate our model. First on training data. fill in the part that calculates `yhat_train5` and we can measure the accuracy on training data:

In [ ]:
### begin your code here (1 line).
yhat_train5 = knn5.predict(X_train)
### end your code here.
accuracy_score(yhat_train5, y_train)

0.8666666666666667

In [ ]:
### begin your code here (1 line).
yhat_train5_2 = knn5_2.predict(X_train2)
### end your code here.
accuracy_score(yhat_train5_2, y_train2)

0.8777777777777778

??.??% Do the same on validation data by predicting `yhat_validation5`:

In [ ]:
### begin your code here (1 line).
yhat_validation5 = knn5.predict(X_validation)
### end your code here.
accuracy_score(yhat_validation5, y_validation)

0.6333333333333333

In [ ]:
### begin your code here (1 line).
yhat_validation5_2 = knn5_2.predict(X_validation2)
### end your code here.
accuracy_score(yhat_validation5_2, y_validation2)

0.6666666666666666

So, we get ??.??% on validation set.

We got our best result with $k=3$ (??.??% > ??.??% > ??.??%), so let's get a final evaluation on our held-out test set. Predict `yhat_test3`:

In [ ]:
### begin your code here (1 line).
yhat_test3 = knn3.predict(X_test)
### end your code here.
accuracy_score(yhat_test3, y_test)

0.7

In [ ]:
### begin your code here (1 line).
yhat_test3_2 = knn3_2.predict(X_test2)
### end your code here.
accuracy_score(yhat_test3_2, y_test2)

0.7333333333333333

The accuracy on test data (??.??%) is both close to performance on validation and training data and is high.

We have a good $k$-NN model now.